In [14]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sbn
import pickle as pkl 
import networkx as nx 
import datetime as dt
pd.set_option('display.max_columns', None)

In [2]:
# Metadata for cell lines (note there is one entry per compound/moa/target/structure combination, so some compounds appear more than once)
druginfo = pd.read_csv('../data/raw/compoundinfo_beta.txt', sep='\t')
druginfo2 = druginfo[lambda x: ~x.target.isna()]

with open('../../../DDS_graph.pkl', 'rb') as f: 
        DDS_graph = pkl.load(f)

with open('../../../PPI_graph.pkl', 'rb') as f: 
    PPI_graph = pkl.load(f)

drugs = DDS_graph['nodelist']
genes = PPI_graph['nodelist']
all2symb = PPI_graph['genemap']

In [3]:
druginfo = druginfo[lambda x: ~x.target.isna()]
druginfo = druginfo.assign(target2 = lambda x: x.target.str.upper().map(all2symb))
druginfo = druginfo[lambda x: x.target2.isin(genes)]
druginfo = druginfo.assign(my_drug_name = lambda x: x.cmap_name.str.lower())
druginfo = druginfo[lambda x: x.my_drug_name.isin(drugs)]
druginfo = druginfo.assign(moa_type = lambda x: x.moa.str.lower().str.split(' '))
druginfo.moa_type = [x[-1] for x in druginfo.moa_type]
druginfo.head()

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases,target2,my_drug_name,moa_type
627,BRD-K42828737,sunitinib,FLT1,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,FLT1,sunitinib,inhibitor
628,BRD-K42828737,sunitinib,FLT3,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,FLT3,sunitinib,inhibitor
629,BRD-K42828737,sunitinib,FLT4,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,FLT4,sunitinib,inhibitor
630,BRD-K42828737,sunitinib,KDR,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,KDR,sunitinib,inhibitor
631,BRD-K42828737,sunitinib,KIT,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,KIT,sunitinib,inhibitor


In [4]:
moa_map = {
 **{
    'agonist':'agonist',
    'inhibitor':'inhibitor',
    'activator':'agonist',
    'antagonist':'inhibitor'
    }, 
 **{k:'uncertain' for k in druginfo.moa_type.unique() if k not in ['agonist', 'inhibitor', 'activator', 'antagonist']}
}

In [5]:
druginfo = druginfo.assign(moa_type2 = lambda x: x.moa_type.map(moa_map))
druginfo = druginfo.assign(prob_agonist=lambda x: (x.moa_type2 == 'agonist')*1., prob_inhibitor=lambda x: (x.moa_type2 == 'inhibitor')*1.)
druginfo.prob_agonist += (druginfo.moa_type2 == 'uncertain')*0.5
druginfo.prob_inhibitor += (druginfo.moa_type2 == 'uncertain')*0.5

assert (druginfo.prob_agonist + druginfo.prob_inhibitor == 1.).all(), 'prob_agonist + prob_inhibitor != 1'
druginfo.head()

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases,target2,my_drug_name,moa_type,moa_type2,prob_agonist,prob_inhibitor
627,BRD-K42828737,sunitinib,FLT1,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,FLT1,sunitinib,inhibitor,inhibitor,0.0,1.0
628,BRD-K42828737,sunitinib,FLT3,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,FLT3,sunitinib,inhibitor,inhibitor,0.0,1.0
629,BRD-K42828737,sunitinib,FLT4,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,FLT4,sunitinib,inhibitor,inhibitor,0.0,1.0
630,BRD-K42828737,sunitinib,KDR,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,KDR,sunitinib,inhibitor,inhibitor,0.0,1.0
631,BRD-K42828737,sunitinib,KIT,KIT inhibitor,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)...,WINHZLLDWRZWRT-ATVHPVEESA-N,sunitinib-malate,KIT,sunitinib,inhibitor,inhibitor,0.0,1.0


In [6]:
druginfo.groupby('moa_type2').count()['target'].sort_values()

moa_type2
uncertain     402
agonist       622
inhibitor    2239
Name: target, dtype: int64

networkx.classes.reportviews.NodeView

In [7]:
# build our graph (directed)
dG = nx.DiGraph()

# add all drug nodes 
_ = [dG.add_node(drug) for drug in drugs]

# add all gene nodes 
_ = [dG.add_node(gene) for gene in genes]

# add drug-protein edges (targets)- edge attribute is probability of moa 
for i,row in druginfo.iterrows(): 
    dG.add_edge(row.my_drug_name, row.target2, agonist=row.prob_agonist, inhibitor=row.prob_inhibitor)


In [18]:
print(f'# nodes in drug->protein graph [expect: {len(genes) + len(drugs)}]:', len(dG))
print(f'# edges in drug->protein graph:', len(dG.edges()))
print(f'network density:', nx.density(dG.to_undirected()))

# nodes in drug->protein graph [expect: 14789]: 14789
# edges in drug->protein graph: 1663
network density: 1.520806619003996e-05


In [9]:
nodeorder = drugs + genes
agonist_adj = nx.adjacency_matrix(dG, nodelist=nodeorder, weight='agonist')
inhibitor_adj = nx.adjacency_matrix(dG, nodelist=nodeorder, weight='inhibitor')

In [10]:
agonist_adj.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
DPI_graph = {'nodelist'         : nodeorder,           # adjacency ordered drug node list 
             'graphs'           : {
                 'agonist'  : agonist_adj,
                 'inhibitor': inhibitor_adj
                },
             'meta'             : 'drug->protein graph based on drug targets. contains two graphs: inhibitor targets & agonist targets (value indicates probability)',
             'creation_date'    :dt.datetime.now().__str__()}   
                                                       # edge prob is attr 
with open('../../../DPI_graph.pkl', 'wb') as f: 
    pkl.dump(DPI_graph, f)